In [89]:
import numpy as np
import matplotlib.pyplot as plt
from scripts.utils import vpt_smape, vpt_nrmse, vpt_mse, smape_rolling
from scipy.spatial.distance import cdist
from dysts.systems import get_attractor_list
from dysts.flows import __dict__ as flow_models
from dysts.analysis import gp_dim
from dysts.metrics import estimate_kl_divergence
import os

In [90]:
## Create a delay embedding
def delay_embedding(data, D=30, tau=1):
    """
    Create a delay embedding of a time series

    Args:
        data: 1D array representing the context trajectory
        D: embedding dimension
        tau: delay used for the embedding

    Returns:
        embedded_data: Delay embedded data

    Usage:
        arr = np.array([1,2,3,4,5,6,7,8,9])
        delay_embedding(arr,5,1)
    """
    N = len(data)
    indices = np.arange(D) * tau + np.arange(N - (D - 1) * tau)[:, None]
    embedded_data = data[indices]
    #print(embedded_data.shape)
    return embedded_data

def embedding_distance(data, D=30, tau=1):
    """
    Calculate the L2 distance in a delay-embedded space

    Args:
        data: 1D array representing the context trajectory
        D: embedding dimension, needs to be large enough
        tau: delay used for the embedding, probably fine to just set to 1

    Returns:
        min_l2_distance: Minimum L2 distance in the embedding space
    """

    # Create delay embeddings
    embedded_data = delay_embedding(data, D=D, tau=tau)

    # Compute distance of other points to the last point in the embedding space using L2 norm
    last_point = embedded_data[-1]
    l2_distance = cdist(embedded_data[:-D * tau], last_point[None, :])
    #l2_distance = cdist(embedded_data[:-5], last_point[None, :])
    #print(l2_distance)

    return l2_distance

In [91]:
def context_parroting_forecast(data, D=30, tau=1, forecast_total_length=300):
    """
    Simple forecasting model based on context parroting.

    Args:
        data (array-like): The input 1D array representing the context trajectory.
        D (int): Context window size for comparing past data (default 30).
        forecast_total_length (int): Desired length of the forecasted output (default 512).

    Returns:
        tuple: 
            - best_index (int): Index of the best-matching context point.
            - min_embedding_distance (float): Minimum L2 distance in the embedding space.
            - forecast (array-like): Forecasted sequence of the specified length.
    """

    #if D * tau >= len(data)-10:
    #    #raise ValueError("D * tau is too large for the data")
    #    D = len(data)//2

    # Calculate embedding distances
    embedding_distances = embedding_distance(data, D=D, tau=tau)
    min_embedding_distance = np.min(embedding_distances)

    # Find the index with the minimum embedding distance
    min_index = np.argmin(embedding_distances)
    best_index = min_index + (D-1)*tau + 1

    # Extract the motif starting from the best-matching index
    motif = data[best_index:-1]
    motif_length = len(motif)

    # Repeat the motif to create a forecast of the desired length
    num_repeats = forecast_total_length // motif_length + 1
    forecast = np.tile(motif, num_repeats)[:forecast_total_length]

    return best_index, min_embedding_distance, forecast

In [92]:
granularity = 30
context_lengths = 2**np.arange(6, 17)
forecast_length = 300
#simulation_length = 10000
simulation_length = 100000
rolling_window = 64
num_ic = 100
D = 5
# Directory to store trajectories
#trajectory_dir = "trajectories"
trajectory_dir = "long_trajectories"
os.makedirs(trajectory_dir, exist_ok=True)

equation_names = get_attractor_list()
equation_names.remove('FluidTrampoline')
equation_names.remove('HyperLu')
equation_names.remove('SprottMore')
equation_names.remove('StickSlipOscillator')
#equation_names = equation_names[:2]

In [93]:
def mse(y_true, y_pred):
    
    mse = np.mean((y_true - y_pred) ** 2)
    return mse

def mse_rolling(ts1, ts2):

    n = min(ts1.shape[0], ts2.shape[0])
    all_mse = list()
    for i in range(1, n+1):
        mse_val = mse(ts1[:i], ts2[:i])
        all_mse.append(mse_val)

    return np.array(all_mse)

def mae(y_true, y_pred):
    
    mae = np.mean(np.abs(y_true - y_pred))
    return mae

def mae_rolling(ts1, ts2):

    n = min(ts1.shape[0], ts2.shape[0])
    all_mae = list()
    for i in range(1, n+1):
        mae_val = mae(ts1[:i], ts2[:i])
        all_mae.append(mae_val)

    return np.array(all_mae)

In [94]:
average_vpt = dict()
median_vpt = dict()
average_smape = dict()
median_smape = dict()

average_vpt_2 = dict()
median_vpt_2 = dict()
average_smape_2 = dict()
median_smape_2 = dict()

average_l2 = dict()
median_l2 = dict()

average_cdim = dict()
median_cdim = dict()
average_kl = dict()
median_kl = dict()

average_mse = dict()
median_mse = dict()
average_mae = dict()
median_mae = dict()

In [95]:
for equation_name in equation_names:

    try:

        # Path to the trajectory file
        trajectory_path = os.path.join(trajectory_dir, f"{equation_name}.npy")

        # Check if the trajectory file already exists
        if os.path.exists(trajectory_path):
            print(f"Loading existing trajectory for {equation_name}...")
            traj = np.load(trajectory_path)
        else:
            # Generate the trajectory
            print(f"Generating trajectory for {equation_name}...")
            model_class = flow_models[equation_name]  # Get the model class dynamically
            model = model_class()  # Instantiate the model
            traj = model.make_trajectory(simulation_length, pts_per_period=granularity)

            # Save the trajectory
            np.save(trajectory_path, traj)
            print(f"Saved trajectory for {equation_name} at {trajectory_path}")

        # normalize the trajectory
        traj = (traj - np.mean(traj, axis=0)) / np.std(traj, axis=0)

        for context_length in context_lengths:

            all_vpt = list() # collecting all vpt for the parroting model
            all_smape_rolling = list() # collecting all smape values across time
            all_vpt_2 = list() # collecting all vpt for the parroting model
            all_smape_rolling_2 = list() # collecting all smape values across time
            all_l2 = list() # collecting all l2 distances for the embedding model
            all_cdim = list() # collecting all correlation dimensions
            all_kl_dist = list() # collecting all kl divergence values
            all_mse_rolling = list()
            all_mae_rolling = list()

            for i in range(0, len(traj)-context_length-forecast_length, rolling_window):
                if i < rolling_window*num_ic:
                    #print(i//rolling_window)
                    traj_context = traj[i:i+context_length,:]
                    traj_true = traj[i+context_length:i+context_length+forecast_length,:]
                    traj_pred_full = np.zeros_like(traj_true)

                    for dim in range(traj_true.shape[1]):

                        best_index, min_l2_distance, traj_pred = context_parroting_forecast(traj_context[:,dim], D=D)
                        traj_pred_full[:,dim] = traj_pred

                        vpt = vpt_smape(traj_pred, traj_true[:,dim]) / granularity
                        smape_val = np.array(smape_rolling(traj_true[:, dim], traj_pred))

                        all_vpt.append(vpt)
                        all_smape_rolling.append(smape_val)

                        all_l2.append(min_l2_distance)

                        mse_val = mse_rolling(traj_true[:, dim], traj_pred)
                        all_mse_rolling.append(mse_val)
                        
                        mae_val = mae_rolling(traj_true[:, dim], traj_pred)
                        all_mae_rolling.append(mae_val)

                    vpt = vpt_smape(traj_pred_full.squeeze(), traj_true.squeeze()) / granularity
                    smape_val = np.array(smape_rolling(traj_true, traj_pred_full))
                    all_vpt_2.append(vpt)
                    all_smape_rolling_2.append(smape_val)
                    
                    kl_dist = estimate_kl_divergence(traj_true, traj_pred_full)
                    if np.isinf(kl_dist):
                        kl_dist = np.nan
                    all_kl_dist.append(kl_dist)

                    cdim_pred = gp_dim(traj_pred_full)
                    cdim_true = gp_dim(traj_true)
                    all_cdim.append(np.array([cdim_pred, cdim_true]))

            average_vpt[(equation_name,context_length)] = np.mean(all_vpt)
            median_vpt[(equation_name,context_length)] = np.median(all_vpt)
            average_smape[(equation_name,context_length)] = np.mean(all_smape_rolling, axis=0)
            median_smape[(equation_name,context_length)] = np.median(all_smape_rolling, axis=0)

            average_vpt_2[(equation_name,context_length)] = np.mean(all_vpt_2)
            median_vpt_2[(equation_name,context_length)] = np.median(all_vpt_2)
            average_smape_2[(equation_name,context_length)] = np.mean(all_smape_rolling_2, axis=0)
            median_smape_2[(equation_name,context_length)] = np.median(all_smape_rolling_2, axis=0)

            average_l2[(equation_name,context_length)] = np.mean(all_l2)
            median_l2[(equation_name,context_length)] = np.median(all_l2)

            average_cdim[(equation_name,context_length)] = np.mean(all_cdim, axis=0)
            median_cdim[(equation_name,context_length)] = np.median(all_cdim, axis=0)
            average_kl[(equation_name,context_length)] = np.nanmean(all_kl_dist)
            median_kl[(equation_name,context_length)] = np.nanmedian(all_kl_dist)

            average_mse[(equation_name,context_length)] = np.mean(all_mse_rolling, axis=0)
            median_mse[(equation_name,context_length)] = np.median(all_mse_rolling, axis=0)
            average_mae[(equation_name,context_length)] = np.mean(all_mae_rolling, axis=0)
            median_mae[(equation_name,context_length)] = np.median(all_mae_rolling, axis=0)
            
            
    except Exception as e:
        print(e)
        print(f"Skipping {equation_name}", flush=True)
        continue
                

Loading existing trajectory for Aizawa...
Loading existing trajectory for AnishchenkoAstakhov...
Loading existing trajectory for Arneodo...
Loading existing trajectory for ArnoldBeltramiChildress...
Loading existing trajectory for ArnoldWeb...
Loading existing trajectory for AtmosphericRegime...
Loading existing trajectory for BeerRNN...
Loading existing trajectory for BelousovZhabotinsky...
Loading existing trajectory for BickleyJet...
Loading existing trajectory for Blasius...
Loading existing trajectory for BlinkingRotlet...
Loading existing trajectory for BlinkingVortex...
Loading existing trajectory for Bouali...
Loading existing trajectory for Bouali2...
Loading existing trajectory for BurkeShaw...
Loading existing trajectory for CaTwoPlus...
Loading existing trajectory for CaTwoPlusQuasiperiodic...
Loading existing trajectory for CellCycle...
Loading existing trajectory for CellularNeuralNetwork...
Loading existing trajectory for Chen...
Loading existing trajectory for ChenLee..

In [96]:
np.save(f'./parrot_statistics_normalized/D={D}_average_vpt.npy', average_vpt)
np.save(f'./parrot_statistics_normalized/D={D}_median_vpt.npy', median_vpt) 
np.save(f'./parrot_statistics_normalized/D={D}_average_smape.npy', average_smape)
np.save(f'./parrot_statistics_normalized/D={D}_median_smape.npy', median_smape)
np.save(f'./parrot_statistics_normalized/D={D}_average_vpt_2.npy', average_vpt_2)
np.save(f'./parrot_statistics_normalized/D={D}_median_vpt_2.npy', median_vpt_2) 
np.save(f'./parrot_statistics_normalized/D={D}_average_smape_2.npy', average_smape_2)
np.save(f'./parrot_statistics_normalized/D={D}_median_smape_2.npy', median_smape_2)
np.save(f'./parrot_statistics_normalized/D={D}_average_cdim.npy', average_cdim)
np.save(f'./parrot_statistics_normalized/D={D}_median_cdim.npy', median_cdim)
np.save(f'./parrot_statistics_normalized/D={D}_average_kl.npy', average_kl)
np.save(f'./parrot_statistics_normalized/D={D}_median_kl.npy', median_kl)
np.save(f'./parrot_statistics_normalized/D={D}_average_mse.npy', average_mse)
np.save(f'./parrot_statistics_normalized/D={D}_median_mse.npy', median_mse)
np.save(f'./parrot_statistics_normalized/D={D}_average_mae.npy', average_mae)
np.save(f'./parrot_statistics_normalized/D={D}_median_mae.npy', median_mae)
np.save(f'./parrot_statistics_normalized/D={D}_average_l2.npy', average_l2)
np.save(f'./parrot_statistics_normalized/D={D}_median_l2.npy', median_l2)